# ResNet Test
I will build deep plain CNN to check how the number of layers affect the accuracy of models. Then, I will implement ResNet to see if it really improves accuracies of different models. 

I will use MNIST dataset for this simple test 

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../MNIST_data', validation_size=0, one_hot=True)

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


Define method to build plain cnn model 

In [39]:
def define_inputs():
    x = tf.placeholder(dtype=tf.float32, shape=[None, 28, 28, 1])
    y_ = tf.placeholder(dtype=tf.float32, shape=[None, 10])
    keep_prob = tf.placeholder(dtype=tf.float32, shape=())
    return x, y_, keep_prob

In [40]:
def define_model(x, n_duplication, n_mid_layers, keep_prob):
    assert n_mid_layers % n_duplication == 0
    
    # 28 x 28 x 1 input 
    # for simplicity, the first layer is always the same
    layer1 = tf.layers.conv2d(x, filters=32, kernel_size=3, padding="same", activation=tf.nn.relu)
    layer1 = tf.layers.max_pooling2d(layer1, pool_size=2, strides=2)
    
    # 14 x 14 x 8
    layer_mid = tf.layers.conv2d(layer1, filters=32, kernel_size=3, padding="same", activation=tf.nn.relu)
    # 14 x 14 x 8
    layer_mid = tf.layers.conv2d(layer_mid, filters=64, kernel_size=3, padding="same", activation=tf.nn.relu)
    
    # 14 x 14 x 16
    # 14 x 14 x 16

    # last pooling
    layer_pooling = tf.layers.max_pooling2d(layer_mid, pool_size=2, strides=2)
    # now, 7 x 7 x 8
    
    # for simplicity, the last layer is always the same 
    layer_flatten = tf.reshape(layer_pooling, shape=[-1, 7*7*64])
    layer_last = tf.layers.dense(layer_flatten, units=1024, activation=tf.nn.relu)
    layer_last = tf.nn.dropout(layer_last, keep_prob=keep_prob)
    logit = tf.layers.dense(layer_last, units=10, activation=None)
    
    return logit

In [43]:
x, y_, keep_prob = define_inputs()
logit = define_model(x, n_duplication=1, n_mid_layers=1, keep_prob=keep_prob)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=logit))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(logit,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(3000):
        batch = mnist.train.next_batch(50)
        imgs = batch[0].reshape(-1, 28, 28, 1)
        labels = batch[1]
        if i%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:imgs, y_: labels, keep_prob: 1})
            print("step %d, training accuracy %g"%(i, train_accuracy))
        train_step.run(feed_dict={x: imgs, y_: batch[1], keep_prob: 0.5})

    print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images.reshape(-1, 28, 28, 1), y_: mnist.test.labels, keep_prob: 1}))

step 0, training accuracy 0.06
step 100, training accuracy 0.88
step 200, training accuracy 0.96
step 300, training accuracy 0.88
step 400, training accuracy 0.98
step 500, training accuracy 0.98
step 600, training accuracy 0.98
step 700, training accuracy 1
step 800, training accuracy 0.96
step 900, training accuracy 0.98
step 1000, training accuracy 0.96
step 1100, training accuracy 0.98
step 1200, training accuracy 1
step 1300, training accuracy 0.98
step 1400, training accuracy 0.94
step 1500, training accuracy 1
step 1600, training accuracy 0.98
step 1700, training accuracy 0.94
step 1800, training accuracy 0.96
step 1900, training accuracy 0.98
step 2000, training accuracy 0.98
step 2100, training accuracy 0.98
step 2200, training accuracy 1
step 2300, training accuracy 0.94
step 2400, training accuracy 1
step 2500, training accuracy 0.98
step 2600, training accuracy 0.96
step 2700, training accuracy 1
step 2800, training accuracy 1
step 2900, training accuracy 1
test accuracy 0.